In [31]:
import struct
from os.path import exists
from os.path import getsize
import csv
import progressbar

channels = {b'B':"Battery Current (A)", b'P': "Air Pressure (P)", b'H':"Relative Humidity (percent)", b'M':"Battery Temperature (C)", b'N':"Internal Temperature (C)", b'T':"External Temperature (C)", b'1':"X Acceleration (g)", b'2':"Y Acceleration (g)", b'3':"Z Acceleration (g)", b'4':"X Mangetometer (uT)", b'5':"Y Mangetometer (uT)", b'6':"Z Mangetometer (uT)", b'7':"X Gyro (deg per sec)", b'8':"Y Gyro (deg per sec)", b'9':"Z Gyro (deg per sec)", b'L':"Latitude", b'O':"Longitude", b'A':"Altitude (m)", b'S':"Speed (m per s)", b'I':"Time (seconds)", b'V':"VOC density (ppb)", b'C':"eCO2 density (ppm)"} 
print("file path?")
filepath = input();
print("file name?")
filename = input();
targetfile = filepath + filename
file = open(targetfile, "rb")

file path?

file name?
LOG


In [32]:
n_cycles = int(getsize(targetfile)/9)
current_cycle = 1

with progressbar.ProgressBar(max_value=n_cycles) as bar:    
    dataIn = file.read(1)
    while dataIn:
        try: # Check if the dataIn is a valid data source
            title = channels[dataIn]
        except: # If not, go to the next byte
            print(dataIn)
            dataIn = file.read(1)
            continue
    
        try: # Try to read the data in to process
            byteLong  = file.read(4)
            byteFloat = file.read(4)
            dataIn = file.read(1)
        except: # If it fails, we have probably reached EOF; break
            break

        # Convert byteLong and byteFloat to an unsigned long and float respectively
        [timestamp] = struct.unpack("L", byteLong)
        [data] = struct.unpack("f", byteFloat)

        # Create CSV file name
        csvfilename = filepath + title + ".csv"
        # Check if it already exists
        # If it doesn't exist, create the file and write the header
        if (not exists(csvfilename)):
            print("New file: " + csvfilename)
            f = open(csvfilename, "a")
            f.write("Time (ms), " + title + "\n")
            f.close()

        # Now that we know it exists and has a header, open it and write the data
        with open(csvfilename, 'a', newline='', encoding='UTF8') as f:
            writer = csv.writer(f)
            row = [str(timestamp), str(data)]
            writer.writerow(row)
            
        # Update the progress bar
        current_cycle += 1
        bar.update(current_cycle)

print("done")

  0% (269 of 3961827) |                  | Elapsed Time: 0:00:00 ETA:   0:24:48

b'q'
b'q'
b'q'
b'q'
b'q'
b'q'
b'q'
b'q'
b'q'
New file: Battery Current (A).csv
New file: Air Pressure (P).csv
New file: Relative Humidity (percent).csv
New file: Battery Temperature (C).csv
New file: Internal Temperature (C).csv
New file: External Temperature (C).csv
New file: X Acceleration (g).csv
New file: Y Acceleration (g).csv
New file: Z Acceleration (g).csv
New file: X Mangetometer (uT).csv
New file: Y Mangetometer (uT).csv
New file: Z Mangetometer (uT).csv
New file: X Gyro (deg per sec).csv
New file: Y Gyro (deg per sec).csv
New file: Z Gyro (deg per sec).csv
New file: Latitude.csv
New file: Longitude.csv
New file: Altitude (m).csv
New file: Speed (m per s).csv
New file: Time (seconds).csv
New file: VOC density (ppb).csv
New file: eCO2 density (ppm).csv


100% (3961827 of 3961827) |##############| Elapsed Time: 0:24:00 Time:  0:24:00


done
